In [1]:
from keras.preprocessing import image
from keras import Input
from keras import layers
from keras.optimizers import Adam
from keras import Model
from keras import losses

import matplotlib.pyplot as plt
# MacOS matplotlib kernel issue
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tejas/anaconda3/lib/python3.7/site-packages/tenso

In [2]:
train_datagen = image.ImageDataGenerator(rescale=1./255)
train_gen = train_datagen.flow_from_directory('.', classes=['anime_face'], target_size = (128, 128), 
                                              batch_size = 32, color_mode='rgb')

Found 140000 images belonging to 1 classes.


In [3]:
def generator_model(noise=(100, 100, 100)):
    gen_input = Input(shape=noise)
    generator = layers.Conv2D(filters=512, kernel_size=5)(gen_input)
    generator = layers.LeakyReLU(0.1)(generator)
    generator = layers.Conv2DTranspose(filters=256, kernel_size=5)(generator)
    generator = layers.BatchNormalization(momentum=0.5)(generator)
    generator = layers.Conv2D(filters=3, kernel_size=5, activation='tanh')(generator)
    
    model = Model(input=gen_input, output=generator)
    model.compile(optimizer=Adam(lr=0.01), loss=losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

In [4]:
def discriminator_model(image_shape=(128, 128, 3)):
    disc_input = Input(shape=image_shape)
    discriminator = layers.Conv2D(filters=128, kernel_size=5)(disc_input)
    discriminator = layers.LeakyReLU(0.1)(discriminator)
    discriminator = layers.Conv2DTranspose(filters=256, kernel_size=5)(discriminator)
    discriminator = layers.BatchNormalization(momentum=0.5)(discriminator)
    discriminator = layers.Conv2D(filters=512, kernel_size=5)(discriminator)
    discriminator = layers.Flatten()(discriminator)
    discriminator = layers.Dense(1, activation='sigmoid')(discriminator)
    
    model = Model(input=disc_input, output=discriminator)
    model.compile(optimizer=Adam(lr=0.01), loss=losses.binary_crossentropy, metrics=['accuracy'])
    
    return model

In [5]:
noise = (100,100,100)
gen_model = generator_model(noise)
gen_model.summary()








_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 100)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 512)       1280512   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 96, 96, 512)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 100, 100, 256)     3277056   
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 256)     1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 3)         19203     
Total params: 4,577,795
Trainable params: 4,577,283
Non-trainable params: 512
_________________________________________________________

/Users/tejas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  if __name__ == '__main__':


In [6]:
disc_model = discriminator_model()
disc_model.trainable = False
disc_model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 128)     9728      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 124, 124, 128)     0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 128, 128, 256)     819456    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 256)     1024      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 124, 124, 512)     3277312   
________________________________________________

/Users/tejas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()
/Users/tejas/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [7]:
gan_gen_input = Input(shape=noise)
gan_gen = gen_model(gan_gen_input)
gan_dis = disc_model(gan_gen)

gan_model = Model(input=gan_gen_input, output=gan_dis)
gan_model.compile(optimizer=Adam(lr=0.01), loss=losses.binary_crossentropy, metrics=['accuracy'])
gan_model.summary()

/Users/tejas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 100, 100)     0         
_________________________________________________________________
model_1 (Model)              (None, 96, 96, 3)         4577795   
_________________________________________________________________
model_2 (Model)              (None, 1)                 11980033  
Total params: 16,557,828
Trainable params: 4,577,283
Non-trainable params: 11,980,545
_________________________________________________________________
